# ETL Workflow — 10,000 Sales Records Dataset

**Goal:** Build a clean, maintainable ETL pipeline that:
- **Extracts** raw sales data from a CSV (downloaded from Kaggle if needed)
- **Transforms** cleans, standardises country names, computes profit margin
- **Loads** the cleaned data into an SQLite database

## Setup

```bash
pip install pandas sqlalchemy kagglehub country-converter
```

In [74]:
# Imports
import pandas as pd
import sqlite3
from pathlib import Path
import kagglehub
from IPython.display import display
import country_converter as coco

# Paths
DB_PATH   = Path("etl_output.db")
CSV_PATH  = Path("10000_Sales_Records.csv")

# Kaggle dataset (handle + file name)
DATASET_HANDLE = "omaradel99/10000-sales-records"
FILE_IN_DATASET = "10000 Sales Records.csv"

## Extract — Download / load the CSV

In [86]:
if not CSV_PATH.exists():
    print("Downloading dataset from Kaggle...")
    path = kagglehub.dataset_download(DATASET_HANDLE)
    # kagglehub returns a folder → find the actual CSV
    downloaded_csv = Path(path) / FILE_IN_DATASET
    df = pd.read_csv(downloaded_csv)
    df.to_csv(CSV_PATH, index=False)
    print(f"Saved locally as {CSV_PATH}")
else:
    df = pd.read_csv(CSV_PATH)
    print(f"Loaded from local file {CSV_PATH}")

print("Shape:", df.shape)
display(df.head())

Loaded from local file 10000_Sales_Records.csv
Shape: (10000, 14)


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Sub-Saharan Africa,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.00
1,Europe,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.50
2,Middle East and North Africa,Pakistan,Vegetables,Offline,C,1/13/2011,141515767,2/1/2011,6515,154.06,90.93,1003700.90,592408.95,411291.95
3,Sub-Saharan Africa,Democratic Republic of the Congo,Household,Online,C,9/11/2012,500364005,10/6/2012,7683,668.27,502.54,5134318.41,3861014.82,1273303.59
4,Europe,Czech Republic,Beverages,Online,C,10/27/2015,127481591,12/5/2015,3491,47.45,31.79,165647.95,110978.89,54669.06


## Transform — 1. Drop rows with missing essential values

In [76]:
key_cols = ['Country', 'Units Sold', 'Unit Price', 'Unit Cost',
            'Total Revenue', 'Total Cost', 'Total Profit']

before = len(df)
df = df.dropna(subset=key_cols)
print(f"Rows before: {before} → after dropping nulls: {len(df)}")

Rows before: 10000 → after dropping nulls: 10000


## Transform — 2. Standardise country names

In [ ]:
cc = coco.CountryConverter()

# Convert to short official name; unmatched → keep original but title-cased
df['Country_std'] = cc.convert(df['Country'], to='name_short', not_found=None)
df['Country_std'] = df['Country_std'].fillna(df['Country'].str.strip().str.title())

# Optional: replace some known aliases manually (U K → United Kingdom, UAE → United Arab Emirates) - can be extended as needed!
replace_map = {
    'U K': 'United Kingdom',
    'UAE': 'United Arab Emirates',
    'United States Of America': 'United States',
    'Czech Republic': 'Czechia'
}
df['Country_std'] = df['Country_std'].replace(replace_map)

print("Sample of country standardisation:")
display(df[['Country', 'Country_std']].drop_duplicates().head(10))

Sample of country standardisation:


,Country,Country_std
0,Chad,Chad
1,Latvia,Latvia
2,Pakistan,Pakistan
3,Democratic Republic of the Congo,DR Congo
4,Czech Republic,Czechia
5,South Africa,South Africa
6,Laos,Laos
7,China,China
8,Eritrea,Eritrea
9,Haiti,Haiti


## Transform — 3. Compute profit margin

In [78]:
df['Profit_Margin'] = df['Total Profit'] / df['Total Revenue']
df['Profit_Margin_Pct'] = (df['Profit_Margin'] * 100).round(2)

# Guard against division-by-zero (should not happen after cleaning but safe)
df['Profit_Margin'] = df['Profit_Margin'].replace([float('inf'), -float('inf')], None)

print("Profit margin example:")
display(df[['Total Revenue', 'Total Cost', 'Total Profit',
            'Profit_Margin', 'Profit_Margin_Pct']].head())

Profit margin example:


,Total Revenue,Total Cost,Total Profit,Profit_Margin,Profit_Margin_Pct
0,2920025.64,2353920.64,566105.00,0.193870,19.39
1,51008.75,34174.25,16834.50,0.330032,33.00
2,1003700.90,592408.95,411291.95,0.409775,40.98
3,5134318.41,3861014.82,1273303.59,0.247999,24.80
4,165647.95,110978.89,54669.06,0.330032,33.00


## Load — Write cleaned data to SQLite

In [79]:
# Keep only the columns we want in the final table (optional)
final_cols = [
    'Region', 'Country', 'Country_std', 'Item Type', 'Sales Channel',
    'Order Priority', 'Order Date', 'Order ID', 'Ship Date',
    'Units Sold', 'Unit Price', 'Unit Cost',
    'Total Revenue', 'Total Cost', 'Total Profit',
    'Profit_Margin', 'Profit_Margin_Pct'
]
df_final = df[final_cols]

conn = sqlite3.connect(DB_PATH)
df_final.to_sql('sales_cleaned', conn, if_exists='replace', index=False)
conn.close()

print(f"Cleaned data successfully written to {DB_PATH}")
print(f"Table name: sales_cleaned   |   Rows: {len(df_final)}")

Cleaned data successfully written to etl_output.db
Table name: sales_cleaned   |   Rows: 10000


## Quick verification query

In [80]:
with sqlite3.connect(DB_PATH) as conn:
    sample = pd.read_sql("SELECT * FROM sales_cleaned LIMIT 5", conn)
display(sample)

# Example aggregation
with sqlite3.connect(DB_PATH) as conn:
    top_countries = pd.read_sql("""
        SELECT Country_std, SUM("Total Profit") AS total_profit
        FROM sales_cleaned
        GROUP BY Country_std
        ORDER BY total_profit DESC
        LIMIT 10
    """, conn)
print("Top 10 countries by total profit:")
display(top_countries)

,Region,Country,Country_std,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,Profit_Margin,Profit_Margin_Pct
0,Sub-Saharan Africa,Chad,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.00,0.193870,19.39
1,Europe,Latvia,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.50,0.330032,33.00
2,Middle East and North Africa,Pakistan,Pakistan,Vegetables,Offline,C,1/13/2011,141515767,2/1/2011,6515,154.06,90.93,1003700.90,592408.95,411291.95,0.409775,40.98
3,Sub-Saharan Africa,Democratic Republic of the Congo,DR Congo,Household,Online,C,9/11/2012,500364005,10/6/2012,7683,668.27,502.54,5134318.41,3861014.82,1273303.59,0.247999,24.80
4,Europe,Czech Republic,Czechia,Beverages,Online,C,10/27/2015,127481591,12/5/2015,3491,47.45,31.79,165647.95,110978.89,54669.06,0.330032,33.00


Top 10 countries by total profit:


,Country_std,total_profit
0,Kiribati,32454798.26
1,Qatar,30861356.79
2,Grenada,30302769.90
3,Taiwan,30044779.98
4,Malawi,29329125.05
5,Botswana,29019726.67
6,Bangladesh,28092720.77
7,Lithuania,28063374.02
8,Burundi,27911293.45
9,Ireland,27807805.05


In [87]:
# Connect to the SQLite database
conn = sqlite3.connect("/Users/lennydufrain/Desktop/Jeevan/ATC/Data Analysis/etl_output.db") # Adjust path as needed

# List all tables
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
display(tables)

# Preview a table
df = pd.read_sql("SELECT * FROM sales_cleaned LIMIT 10;", conn)
display(df)


,name
0,sales_cleaned


,Region,Country,Country_std,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit,Profit_Margin,Profit_Margin_Pct
0,Sub-Saharan Africa,Chad,Chad,Office Supplies,Online,L,1/27/2011,292494523,2/12/2011,4484,651.21,524.96,2920025.64,2353920.64,566105.00,0.193870,19.39
1,Europe,Latvia,Latvia,Beverages,Online,C,12/28/2015,361825549,1/23/2016,1075,47.45,31.79,51008.75,34174.25,16834.50,0.330032,33.00
2,Middle East and North Africa,Pakistan,Pakistan,Vegetables,Offline,C,1/13/2011,141515767,2/1/2011,6515,154.06,90.93,1003700.90,592408.95,411291.95,0.409775,40.98
3,Sub-Saharan Africa,Democratic Republic of the Congo,DR Congo,Household,Online,C,9/11/2012,500364005,10/6/2012,7683,668.27,502.54,5134318.41,3861014.82,1273303.59,0.247999,24.80
4,Europe,Czech Republic,Czechia,Beverages,Online,C,10/27/2015,127481591,12/5/2015,3491,47.45,31.79,165647.95,110978.89,54669.06,0.330032,33.00
5,Sub-Saharan Africa,South Africa,South Africa,Beverages,Offline,H,7/10/2012,482292354,8/21/2012,9880,47.45,31.79,468806.00,314085.20,154720.80,0.330032,33.00
6,Asia,Laos,Laos,Vegetables,Online,L,2/20/2011,844532620,3/20/2011,4825,154.06,90.93,743339.50,438737.25,304602.25,0.409775,40.98
7,Asia,China,China,Baby Food,Online,C,4/10/2017,564251220,5/12/2017,3330,255.28,159.42,850082.40,530868.60,319213.80,0.375509,37.55
8,Sub-Saharan Africa,Eritrea,Eritrea,Meat,Online,L,11/21/2014,411809480,1/10/2015,2431,421.89,364.69,1025614.59,886561.39,139053.20,0.135580,13.56
9,Central America and the Caribbean,Haiti,Haiti,Office Supplies,Online,C,7/4/2015,327881228,7/20/2015,6197,651.21,524.96,4035548.37,3253177.12,782371.25,0.193870,19.39
